In [1]:
from datetime import date

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# Extract

In [3]:
dim_medico = pd.read_sql_table('medico', co_sa)


# Transformations

In [4]:
dim_medico.info()

In [5]:
dim_medico.describe(include='all')

In [6]:
 dim_medico.replace({np.nan: 'no aplica', ' ': 'no aplica','':'no_aplica'}, inplace=True)
dim_medico["saved"] = date.today()

# load

In [7]:
dim_medico.head()

In [8]:
dim_medico.to_sql('dim_medico', etl_conn, if_exists='replace',index_label='key_dim_medico')